In [1]:
# To create an excel file for Controlling you need to fetch DIM_all_auction & FACT_all_transactions tables from 063_Revenue recognition report with Dax Studio

In [2]:
import pandas as pd
from openpyxl import load_workbook

In [3]:
files_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/PowerBI/05.23/30,05/new logic/all/16_06_23/dax_studio/'
save_dir = 'C:/Users/User/Desktop/Moje dokumenty/Raporty obce/Kacper/63_issue_158946_payments/Excel/'

In [4]:
DIM_all_auction = pd.read_csv(files_dir + 'DIM_all_auction.csv')

In [5]:
FACT_all_transactions = pd.read_csv(files_dir + 'FACT_all_transactions.csv')

C:\Users\User\AppData\Local\Temp\ipykernel_4216\256377864.py:1: DtypeWarning: Columns (2,14) have mixed types. Specify dtype option on import or set low_memory=False.
  FACT_all_transactions = pd.read_csv(files_dir + 'FACT_all_transactions.csv')


In [6]:
DIM_all_auction = DIM_all_auction[['Main auction', 'Company', 'Open Amount (CHF)22', 'Orders_not_sent']]
DIM_all_auction = DIM_all_auction.rename(columns={'Open Amount (CHF)22': 'open_amount(CHF)'})

In [7]:
DIM_all_auction.drop_duplicates(subset='Main auction', inplace=True)

In [8]:
FACT_all_transactions = FACT_all_transactions[['Main auction', 'Payment_id/Ticket_id', 'Account', 'Selling_account', 'Refund?[0-no,1-del_auf,2-ticket]', 'Negative amount?', 'Create_date', 'Transaction_Date', 'Amount_Brutto(local)', 'Amount_Netto(local)', 'Vat_percent', 'Amount_netto(EUR)', 'Sum_Amount_Paid(CHF)22', 'Amount_netto(CHF)_web', 'Shipped_date', 'First_Payment']]
FACT_all_transactions = FACT_all_transactions.rename(columns={'Sum_Amount_Paid(CHF)22': 'Amount_netto(CHF)'})

In [9]:
merged = pd.merge(FACT_all_transactions, DIM_all_auction, on='Main auction', how='left')

In [10]:
# Clean data
merged['Selling_account'] = merged['Selling_account'] .astype(str)
merged['Selling_account'] = merged['Selling_account'] .astype(str).replace(r'\.0', '', regex=True)
merged['Shipped_date'] = pd.to_datetime(merged['Shipped_date'])
merged['First_Payment'] = pd.to_datetime(merged['First_Payment'])
merged['Create_date'] = pd.to_datetime(merged['Create_date'])
merged['Transaction_Date'] = pd.to_datetime(merged['Transaction_Date'])

In [11]:
# To create columns with year only
merged['First_Payment_Year'] = merged['First_Payment'].dt.year
merged['Create_date_Year'] = merged['Create_date'].dt.year
merged['Transaction_Date_Year'] = merged['Transaction_Date'].dt.year
merged['Shipped_date_Year'] = merged['Shipped_date'].dt.year


In [12]:
unique_companies = merged['Company'].unique()

In [13]:
writer = pd.ExcelWriter(save_dir + 'output.xlsx', engine='xlsxwriter')

for company in unique_companies:
    # Create a dataframe name by replacing spaces with underscores
    dataframe_name = company.replace(' ', '_')
    
    # Filter the merged dataframe based on the company name
    filtered_df = merged[merged['Company'].str.contains(company, na=False)].copy()
    
    # Assign the filtered dataframe to a dynamically created variable with the dataframe name
    globals()[dataframe_name] = filtered_df

    # Check if the company is "Beliani_DE". It has to be excluded from writing since we have to split it to fit into excel
    if company == "Beliani DE":
        continue  # Skip writing the dataframe

    # Write the dataframe to a sheet named after the company in the Excel file
    filtered_df.to_excel(writer, sheet_name=dataframe_name, index=False)

# Save the Excel file
writer.save()

C:\Users\User\AppData\Local\Temp\ipykernel_4216\4103503056.py:21: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [14]:
# Split skipped Beliani_DE into two dataframes
Beliani_DE23 = Beliani_DE[(Beliani_DE['Shipped_date'] >= pd.Timestamp('2023-01-01'))]
Beliani_DE22 = Beliani_DE[(Beliani_DE['Shipped_date'] < pd.Timestamp('2023-01-01'))]

In [15]:
output_file = save_dir + 'output.xlsx'

try:
    book = load_workbook(output_file)
except FileNotFoundError:
    book = Workbook()

writer = pd.ExcelWriter(output_file, engine='openpyxl')
writer.book = book

Beliani_DE23.to_excel(writer, sheet_name='Beliani_DE23', index=False)
Beliani_DE22.to_excel(writer, sheet_name='Beliani_DE22', index=False)

writer.save()

C:\Users\User\AppData\Local\Temp\ipykernel_4216\4169855882.py:9: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
C:\Users\User\AppData\Local\Temp\ipykernel_4216\4169855882.py:14: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


MemoryError: 